In [1]:
from songs import *

# НЕЙРОНКИ

In [2]:
import numpy as np
from numpy import random
import sys
from graphviz import Digraph

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
MEMORY_TIMES = 64
STARTER = 8

ADDITIONAL_MEMORY = 8

MEMORY_SIZE = MEMORY_TIMES * 13 + ADDITIONAL_MEMORY

In [4]:
NOTE_NAMES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B", "C^"]
NAMES = []
for i in range(8):
    NAMES.append("[tb" + str(i) + "]")
for t in range(-MEMORY_TIMES, 0):
    for note_name in NOTE_NAMES:
        NAMES.append(note_name + str(t))

In [5]:
TURNED = []  # список включившихся нейронов
SUCCESS = []  # список нейронов, поучавствовавших в верном ответе
class Neuron:
    def __init__(self, name, oper, layer, isnot=False):
        self.name = name
        self.layer = layer  # номер слоя
        
        self.operation = oper
        self.outputs = []  # список нейронов-выходов
        self.cnt = 0       # подсчёт пришедшего числа сигналов
        self.n = isnot     # инвертирует ли выход
        
        self.inputs = []   # входные связи и их оценки
        self.rewards = []
        self.precedents = []
        
        self.emits = 1
        self.used = 2
        
    def signal(self):
        self.cnt += 1        
        
    def emit(self):
        self.used += 1
        if self.operation == "conj":
            res = self.cnt == len(self.inputs)
        else:
            res = self.cnt > 0
        
        if ((not self.n and res) or (self.n and res)):  # включение нейрона
            self.emits += 1
            TURNED.append(self)
            
            for neuron in self.outputs:
                neuron.signal()
                
    def reward(self, true_val):  # оценивание связей
        if self.n:
            true_val = not true_val  # нужно ли было выключаться
            
        decision = self in TURNED  # был ли включён этот нейрон
        
        #if decision == true_val:  # если выдал нужный сигнал, добавляем в список молодцев
        #    SUCCESS.append(self)
        
        for i, inp in enumerate(self.inputs):
            my_val = inp in TURNED  # входной сигнал
            if my_val == decision:  # интересная эвристика: оцениваем только когда решение было за нейроном
                self.precedents[i][my_val] += 1
                if my_val == true_val:
                    self.rewards[i][my_val] += 1
                inp.reward(true_val)  # переходим во вход рекурсивно

In [6]:
class Func:
    def __init__(self, agent, layers, verbose=False):
        self.layers = layers
        self.neurons = [Neuron(NAMES[i], "disj", 0) for i in range(MEMORY_SIZE)]  # входы
        for i in range(13):                                               # выходы
            self.neurons.append(Neuron(NOTE_NAMES[i] + "outp", "disj", layers)) 
        self.ids = len(self.neurons)       
        
        self.Random = np.random.RandomState(seed=179)        
        self.verbose = verbose
        self.depth = ""
        
        self.agent = agent
        
    def apply(self, x):  # проход по нейросети
        global TURNED    # сначала всё выключаем
        global SUCCESS
        TURNED = []
        SUCCESS = []
        for neuron in self.neurons:
            neuron.cnt = 0
            
        for i, val in enumerate(x):  # посылаем сигналы на нужные входы
            if val == 1:
                self.neurons[i].signal()
                
        for layer in range(0, self.layers + 1):  # проходимся по слоям
            for N in self.neurons:
                if N.layer == layer:
                    N.emit()
        
        # собираем включившиеся выходы
        return np.array([self.neurons[MEMORY_SIZE + i] in TURNED for i in range(13)])
    
    def reward(self, true_vals):  # награждаем связи
        for i, val in enumerate(true_vals):
            self.neurons[i + MEMORY_SIZE].reward(val)
    
    def randomOne(self, layer):
        # выбираем случайный включившийся нейрон
        candidates = []
        proba = []
        for N in TURNED:
            if N.layer < layer:
                candidates.append(N)
                proba.append(1 - N.emits / N.used)
        
        proba = np.array(proba)
        proba /= proba.sum()
        return self.Random.choice(candidates, p=proba)
    def connect(self, N1, N2, rewards=[1, 2], precedents=[2, 3]):
        # соединяем два нейрона новой связью; rewards-precedents это её исходная оценка
        if N1 in N2.inputs:
            if self.verbose:
                print(self.depth, "tried to create existing edge, procedure failed")
            return False
        
        if self.verbose:
            print(self.depth, "new edge between layers: ", N1.name, "(", N1.layer, ")", " / ", N2.name, "(", N2.layer, ")")
        N1.outputs.append(N2)
        N2.inputs.append(N1)
        N2.rewards.append(copy.copy(rewards))
        N2.precedents.append(copy.copy(precedents))
        return True
    def check_for_blow(self, N):
        # проверяем нейрон; если у него нет входов или выходов, взрываем
        
        if not N in self.neurons: #возможно, он уже взорван
            return True
        if N.layer == 0 or N.layer == self.layers: # возможно, это вход или выход
            return False
        
        if len(N.outputs) == 0:
            if self.verbose:
                print(self.depth, "neuron destroyed! ", N.name, "(", N.layer, ")")
            for inp in N.inputs:
                self.disconnect(inp, N)  # возможны рекурсивные взрывы
        elif len(N.inputs) == 0:
            if self.verbose:
                print(self.depth, "neuron destroyed! ", N.name, "(", N.layer, ")")
            for outp in N.outputs:
                self.disconnect(N, outp)  # возможны рекурсивные взрывы
        else:
            return False  # взрыва не произошло
            
        # при взрыве удаляем нейрон из списка включившихся и из списка нейронов
        if N in TURNED:
            TURNED.remove(N)
        if N in self.neurons:
            self.neurons.remove(N)
        return True
    def disconnect(self, N1, N2):
        # разъединяем связь
        if self.verbose:
            print(self.depth, "remove edge between layers: ", N1.name, "(", N1.layer, ")", " / ", N2.name, "(", N2.layer, ")")
        N1.outputs.remove(N2)
        del N2.rewards[N2.inputs.index(N1)]
        del N2.precedents[N2.inputs.index(N1)]
        N2.inputs.remove(N1)
        
        # какие-то нейроны могли остаться без входов или выходов и их нужно взорвать        
        self.check_for_blow(N1)
        self.check_for_blow(N2)
        
        '''  проверка на o-o-o, пока не заработало
        if len(N1.inputs) == 1 and len(N1.outputs) == 1 and not N1.n:
            self.connect(N1.inputs[0], N1.outputs[0], N1.rewards[0], N1.precedents[0])
            self.disconnect(N1.inputs[0], N1)
            #self.disconnect(N1, N1.outputs[0])
        if len(N2.inputs) == 1 and len(N2.outputs) == 1 and not N2.n:
            self.connect(N2.inputs[0], N2.outputs[0], N2.rewards[0], N2.precedents[0])
            self.disconnect(N2.inputs[0], N2)
            #self.disconnect(N2, N2.outputs[0])
        '''  
        
    def add_reason(self, conseq):
        # добавление причины
        if conseq.layer <= 1:
            if self.verbose:
                print(self.depth, "no place for reason :(")
            return None
        
        reason = Neuron("rsn" + str(self.ids), "disj", self.Random.randint(1, conseq.layer), True)
        self.neurons.append(reason)
        self.ids += 1
        
        if self.verbose:
            print(self.depth, "created reason ", reason.name, " (", reason.layer, ")")
        
        self.connect(reason, conseq)        
        return reason
    def add_stub(self, cause, conseq, stublayer):
        # добавление заглушки
        if conseq.layer <= 1 or conseq.layer - cause.layer <= 1:
            if self.verbose:
                print(self.depth, "no place for stub :(")
            return None
        
        stub = Neuron("stub" + str(self.ids), "conj", stublayer)
        self.neurons.append(stub)
        self.ids += 1
        
        if self.verbose:
            print(self.depth, "created stub ", stub.name, " (", stublayer, ")")
        
        # оценки связей берём из разорванной связи
        r, p = conseq.rewards[conseq.inputs.index(cause)], conseq.precedents[conseq.inputs.index(cause)]
        self.connect(cause, stub, r, p)
        self.connect(stub, conseq, r, p)
        self.disconnect(cause, conseq)
                
        return stub
        
    def turn(self, N, pos):
        # АЛГОРИТМ ПОПРАВКИ!
        self.depth += "    "
        
        # Если мы оказались на входе, то сразу говорим, что ничего не вышло
        if N.layer == 0:
            if self.verbose:
                print(self.depth, N.name, " is input neuron, procedure failed")
            self.depth = self.depth[:-4]
            return False
        
        # если этот нейрон выдал правильный сигнал хоть для какого-то выхода, его не трогаем
        if N in SUCCESS:
            if self.verbose:
                print(self.depth, N.name, " is success neuron, procedure failed")
            self.depth = self.depth[:-4]
            return False
            
        
        if self.verbose:
            print(self.depth, "turning ", pos, ": ", N.name)
        
        # Если из нейрона выходит отрицание, то нам нужно получить значение, обратное к pos
        if N.n:
            pos = not pos
            if self.verbose:
                print(self.depth, "actualy ", pos, ": ", N.name)
        
        
        
        if N.operation == "disj" and pos == 1:
            one = self.randomOne(N.layer)
            self.connect(one, N)
        elif N.operation == "conj" and pos == 0:
            reason = self.add_reason(N)
            if not reason is None:
                self.turn(reason, pos)
        else:
            error = 0
            for taken, rew, pr in zip(N.inputs, N.rewards, N.precedents):
                if taken in N.inputs and (taken in TURNED) != pos:
                    error += 1
                    
            for taken, rew, pr in zip(N.inputs, N.rewards, N.precedents):
                if taken in N.inputs and (taken in TURNED) != pos:
                    features = [error, len(N.inputs), N.layer, taken.layer,
                                rew[int(not pos)] / pr[int(not pos)], rew[int(pos)] / pr[int(pos)]]

                    a = self.agent.get_action(features)
                    
                    if a == 0:            
                        self.turn(taken, pos)
                    elif a == 1:
                        if taken.layer + 1 < N.layer:
                            stublayer = self.Random.randint(taken.layer + 1, N.layer)             
                            stub = self.add_stub(taken, N, stublayer)

                            if pos == 0:
                                stub.cnt += 1
                            self.turn(stub, pos)
                    elif a == 2:
                        self.disconnect(taken, N)

                    # нейрон, в котором мы сейчас сидим, может взорваться по цепочке;
                    # в таком случае, алгоритм типауспешно заканчивается
                    if not N in self.neurons:
                        if self.verbose and not result:
                            print(self.depth, "we blowed up!")
                        self.depth = self.depth[:-4]
                        return

        self.depth = self.depth[:-4]
        return
    
    def draw(self):
        # рисуем нейросеть
        dot = Digraph(format='png')
        for N in self.neurons:
            if N.layer == 0:
                dot.attr('node', shape='doublecircle')
            elif N.layer == self.layers:
                dot.attr('node', shape='circle')
            elif N.operation == "conj":
                dot.attr('node', style='filled', color='red')
            else:
                dot.attr('node', style='filled', color='lightblue')
            dot.node(N.name)
        for N in self.neurons:
            for inp, r, p in zip(N.inputs, N.rewards, N.precedents):
                dot.edge(inp.name, N.name, label = "{0:.2f}".format(r[0] / p[0])  + "/" + "{0:.2f}".format(r[1] / p[1]))
        #dot.render(view=True)
        return dot

In [18]:
from sklearn.neural_network import MLPClassifier
class Agent:
    def __init__(self):
        self.history_f = []
        self.history_a = []
        self.NN = MLPClassifier(hidden_layer_sizes=(20,20),
                      activation='tanh',
                      warm_start=True, #keep progress between .fit(...) calls
                      max_iter=1 #make only 1 iteration on each .fit(...)
                     )
        self.NN.fit([[0,0,0,0,0,0]]*3, range(3));
        self.alpha = 1
    
    def get_action(self, f):
        proba = self.NN.predict_proba([f])[0]
        basic_proba = self.alpha * np.ones((3)) / 3.0 + (1 - self.alpha) * proba
        #print(proba)
        a = np.random.choice(range(3), p=basic_proba)
        
        self.history_f.append(f)
        self.history_a.append(a)        
        return a
    
    def learn(self, hf, ha):
        self.NN.fit(hf, ha)

In [19]:
class Player:
    def __init__(self, agent):
        self.play_note = Func(agent, 6)
           
    def learn_to_play(self, song_to_learn, verbose=False, learn=True, limit = -1):
        if limit == -1:
            limit = len(song_to_learn.notes)
        self.play_note.verbose = verbose
        
        # исходная память
        memory = np.zeros((MEMORY_SIZE), dtype=int)
        memory[-STARTER*13:] = song_to_learn.notes[:STARTER].flatten()

        t = STARTER
        memory[t % 8] = 1
        
        errors = 0
        
        while t < limit:    
            # играем
            output = self.play_note.apply(memory)
            # награждаем связи
            self.play_note.reward(song_to_learn.notes[t])
            
            if verbose:
                print(output.astype(int))
            
            # ищем ошибки
            for note in range(13):    
                if output[note] != song_to_learn.notes[t][note]:
                    errors += 1
                    if learn:  # дообучаемся: говорим исправиться на этом входе
                        if output[note] == 0:
                            self.play_note.turn(self.play_note.neurons[note + MEMORY_SIZE], 1)
                        else:
                            self.play_note.turn(self.play_note.neurons[note + MEMORY_SIZE], 0)
                    #return t
            
            # переходим к следующей итерации
            memory = np.concatenate([np.zeros((ADDITIONAL_MEMORY)), memory[ADDITIONAL_MEMORY + 13:], song_to_learn.notes[t]])
            t += 1
            memory[t % 8] = 1
            
        return errors
    
    def improvise(self, starter, length=128):  # играет чё хочет
        memory = np.zeros((MEMORY_SIZE), dtype=int)
        memory[-STARTER*13:] = starter.notes[:STARTER].flatten()

        result = MySong(starter.notes[:STARTER])
        
        t = STARTER
        memory[t % 8] = 1
        while t < length:    
            output = self.play_note.apply(memory)
            result.add(output)

            memory = np.concatenate([np.zeros((ADDITIONAL_MEMORY)), memory[ADDITIONAL_MEMORY + 13:], output])
            t += 1
            memory[t % 8] = 1
        result.finish()
        
        return result

## Обучение

Создаём нового игрока и грузим пока кузнечика

In [20]:
player = Player(Agent())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [21]:
kuznechik = Song('../basic midi/track (1).mid')

In [59]:
player.learn_to_play(kuznechik, verbose=True, limit=21)

[0 0 0 0 0 0 0 0 0 0 0 0 0]
     turning  1 :  Aoutp
     new edge between layers:  E-2 ( 0 )  /  Aoutp ( 6 )
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
     turning  1 :  G#outp
     new edge between layers:  A-10 ( 0 )  /  G#outp ( 6 )
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
     turning  1 :  G#outp
     new edge between layers:  G#-2 ( 0 )  /  G#outp ( 6 )
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0]
     turning  0 :  G#outp
         A-10  is input neuron, procedure failed
         G#-2  is input neuron, procedure failed
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
     turning  1 :  G#outp
     new edge between layers:  G#-6 ( 0 )  /  G#outp ( 6 )
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0]
     turning  1 :  Eoutp
     new edge between layers:  A-14 ( 0 )  /  Eoutp ( 6 )
     turning  0 :  G#outp
         A-10  is input neuron, procedure failed
         G#-2  is input neuron, procedure failed
         G#-6  is input 

9

In [64]:
player.play_note.draw()

Будем добиваться, чтобы он правильно сыграл первые n долей, и постепенно увеличивать n... Текущая версия застревает на 67 - когда требуется нажать новую ноту, и нужно построить такое правило, которое включалось бы только на последней итерации

In [19]:
limit = 8
neurons = [MEMORY_SIZE + 13]

while limit < 128:#len(kuznechik.notes):
    print("limit: ", limit, " neurons_at_start: ", len(player.play_note.neurons))
    errors = [kuznechik.notes[:limit].sum()]
    while errors[-1] > 0:
        errors.append(player.learn_to_play(kuznechik, limit=limit))
        neurons.append(len(player.play_note.neurons))
        print(errors[-1], end=" ")
        sys.stdout.flush()
    print("")
    limit += 1

limit:  8  neurons_at_start:  857
0 
limit:  9  neurons_at_start:  857
1 0 
limit:  10  neurons_at_start:  857
0 
limit:  11  neurons_at_start:  857
1 0 
limit:  12  neurons_at_start:  857
0 
limit:  13  neurons_at_start:  857
2 2 2 1 1 0 
limit:  14  neurons_at_start:  857
0 
limit:  15  neurons_at_start:  857
1 1 0 
limit:  16  neurons_at_start:  853
0 
limit:  17  neurons_at_start:  853
0 
limit:  18  neurons_at_start:  853
0 
limit:  19  neurons_at_start:  853
1 0 
limit:  20  neurons_at_start:  853
0 
limit:  21  neurons_at_start:  853
1 2 1 2 3 2 3 2 2 1 3 2 1 2 1 3 1 1 1 2 2 2 2 2 1 1 2 1 2 1 2 2 1 2 2 2 1 2 1 2 1 2 1 2 1 2 1 1 1 2 1 2 1 2 2 2 1 2 3 2 2 3 2 3 3 3 3 2 3 1 1 3 2 2 3 2 2 2 2 1 3 2 3 1 1 3 3 2 1 2 2 1 2 1 2 2 2 3 1 1 2 1 1 2 1 2 2 1 2 1 1 2 2 2 1 2 1 1 1 2 2 2 2 1 2 2 2 2 2 1 1 2 2 1 3 2 2 2 1 1 2 3 2 1 1 2 2 2 1 2 3 2 3 1 2 2 2 1 2 2 3 2 1 2 3 2 3 3 2 2 2 1 2 1 1 1 3 2 3 2 1 2 2 2 1 2 2 1 2 1 2 2 3 3 3 3 3 1 3 2 1 1 2 2 1 2 1 2 1 3 2 3 3 2 3 2 2 2 1 1 1 1 1 1 1 2 3

KeyboardInterrupt: 

А вот стандартная процедура (несколько раз проходим по выборке). Ошибка сначала падает, потом застревает. Количество нейронов стабильно растёт.

In [69]:
player = Player()

TypeError: __init__() missing 1 required positional argument: 'agent'

In [141]:
errors = [kuznechik.notes.sum()]
neurons = [MEMORY_SIZE + 13]
while errors[-1] > 0:
    errors.append(player.learn_to_play(kuznechik))
    neurons.append(len(player.play_note.neurons))
    print(errors[-1], end=" ")
    sys.stdout.flush()
    
    #if errors[-1] > errors[-2]:
    #    player.play_note.learning_rate += 1

94 80 77 78 81 78 83 74 92 81 76 82 63 76 64 83 74 84 70 72 70 80 71 67 74 66 

KeyboardInterrupt: 

In [ ]:
plt.plot(errors)  # график количества ошибок в эпоху

In [ ]:
plt.plot(neurons)  # график количества нейронов в эпоху

In [ ]:
player.play_note.draw()

In [18]:
player.play_note.draw().render('fail', view=True) 

'fail.png.png'

СНАЧАЛА БЫ ВЫУЧИТЬ КУЗНЕЧИКА!

In [24]:
agent = Agent()
player = Player(agent)
prev_error = player.learn_to_play(kuznechik, learn=False)

def session():
    agent.history_f = []
    agent.history_a = []
    for i in range(1):
        player.learn_to_play(kuznechik)
    
    errors = player.learn_to_play(kuznechik, learn=False)
    print(errors, end=" ")
    
    global prev_error
    improvement = errors / prev_error
    prev_error = errors
    return agent.history_f, agent.history_a, improvement

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [26]:
n_samples = 30
percentile = 70


for i in range(1900):
    #generate new sessions
    sessions = [session() for i in range(n_samples)]
    player.play_note.agent.alpha *= 0.97
    print("")

    batch_states,batch_actions,batch_rewards = map(np.array,zip(*sessions))
    #batch_states: a list of lists of states in each session
    #batch_actions: a list of lists of actions in each session
    #batch_rewards: a list of floats - total rewards at each session

    threshold = np.percentile(batch_rewards, percentile)
    
    elite_states = batch_states[batch_rewards > threshold]
    elite_actions = batch_actions[batch_rewards > threshold]
    
    elite_states, elite_actions = map(np.concatenate,[elite_states,elite_actions])
    #elite_states: a list of states from top games
    #elite_actions: a list of actions from top games
    
    agent.learn(elite_states, elite_actions)

    print("mean reward = %.5f\tthreshold = %.1f\tneurons = %.1f"%(np.mean(batch_rewards),threshold, len(player.play_note.neurons)))

74 67 53 60 59 71 77 69 69 79 77 63 65 68 58 67 59 71 86 74 89 73 80 63 77 84 78 60 75 88 
mean reward = 1.01766	threshold = 1.1	neurons = 1486.0
79 81 86 79 69 74 69 78 81 63 76 67 68 62 69 64 70 72 131 76 73 65 75 75 76 96 70 89 66 66 
mean reward = 1.01096	threshold = 1.1	neurons = 1559.0
64 70 73 121 53 66 59 59 76 64 58 51 58 73 79 70 71 77 63 63 67 68 65 67 72 68 67 66 50 67 
mean reward = 1.02300	threshold = 1.1	neurons = 1494.0
69 73 62 62 58 66 66 73 59 50 61 69 64 51 64 66 60 61 60 60 60 52 54 51 59 76 64 68 62 63 
mean reward = 1.00568	threshold = 1.0	neurons = 1532.0
77 94 65 70 81 62 72 63 67 59 65 64 75 73 94 71 76 92 62 65 77 65 99 72 72 60 83 64 86 61 
mean reward = 1.02374	threshold = 1.2	neurons = 1498.0
75 74 56 102 56 58 70 53 69 79 89 72 65 72 64 69 80 65 65 65 78 67 92 103 91 71 68 70 67 73 
mean reward = 1.03081	threshold = 1.1	neurons = 1561.0
83 66 60 71 61 66 71 74 100 60 55 54 58 63 64 66 68 64 66 59 62 71 61 67 63 66 57 74 78 64 
mean reward = 1.00718	thresh

99 97 93 114 85 87 93 98 92 104 96 94 90 91 93 92 91 96 97 86 92 88 92 90 104 106 110 104 96 88 
mean reward = 1.00092	threshold = 1.0	neurons = 3361.0
89 90 102 108 113 108 112 121 108 111 98 106 109 108 103 109 

KeyboardInterrupt: 

In [ ]:
elite_actions

Проверим, что всё окей. Подадим на вход начало кузнечика, а дальше пусть играет сам (память занимается тем, что игрок сам нажимает на рояле). Поскольку так заданы правила, кузнечик будет воспроизведён точно

In [ ]:
player.improvise(kuznechik).play()

Окей, чтобы услышать что-то новое, нужно другое начало. Возьмём его из другой песенки

In [65]:
simple_song = Song([12, -1, 11, -1, 9, -1, 7, -1])
result = player.improvise(simple_song)
result.play()

In [66]:
simple_song = Song('../basic midi/track (2).mid')
result = player.improvise(simple_song)

Первый шедевр:

In [67]:
result.play()

Игрок пока использует только несколько нот. Просто в кузнечике есть не все 13 нот, так что мы можем его посмещать, чтобы получить новую информацию. Попробуем сместить кузнечика на одну ноту

In [130]:
player = Player()

In [131]:
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))

In [133]:
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('../basic midi/track (' + str(i) + ').mid'))

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [134]:
import random
random.shuffle(Songs)
prev_errors = np.array([1000 for i in range(len(Songs))])

In [135]:
while prev_errors.sum() > 100:
    errors = []
    for song in Songs:
        errors.append(player.learn_to_play(song))
        print(errors[-1], end=" ")
        sys.stdout.flush()
    print("")
    
    prev_errors = np.array(errors)
    print("sum = ", prev_errors.sum())

39 64 65 65 42 50 49 70 44 48 48 38 44 55 51 62 50 51 51 47 71 0 0 57 58 42 53 76 40 53 56 42 53 55 42 50 51 58 42 43 52 38 40 26 0 57 55 45 42 48 52 54 65 50 72 49 34 60 44 49 54 36 40 60 48 51 41 52 47 
sum =  3336
35 51 58 57 35 50 39 73 43 47 46 38 43 48 51 60 50 48 50 47 73 0 0 52 59 40 50 73 37 52 56 41 49 53 42 46 51 58 42 40 53 38 40 26 0 58 54 48 41 44 56 51 68 47 72 49 34 58 42 48 54 36 42 59 47 51 40 51 46 
sum =  3236
35 50 57 56 35 50 41 71 41 46 46 39 41 51 51 60 50 45 50 44 73 0 0 52 56 39 52 72 37 51 55 39 51 54 40 44 50 56 42 42 53 38 39 26 0 59 57 44 42 46 54 50 66 50 71 50 35 56 43 48 53 38 41 57 48 51 40 50 46 
sum =  3215
35 53 53 53 34 51 38 72 42 46 46 38 41 50 47 58 51 46 50 45 70 0 0 50 58 38 49 72 37 50 54 40 53 53 40 49 50 57 41 40 50 37 41 26 0 54 55 46 41 46 55 50 65 48 68 46 35 59 42 45 52 36 41 58 46 50 38 52 45 
sum =  3177
36 50 57 52 36 51 39 68 40 45 45 38 40 49 50 59 49 47 49 43 74 0 0 50 56 39 51 72 37 51 55 40 51 52 40 50 50 58 38 41 53 36 41 26 0 

KeyboardInterrupt: 

In [136]:
len(player.play_note.neurons)

1736

In [391]:
simple_song = Song('test/whomadethis.mid')
result = player.improvise(simple_song)
result.play()

KeyboardInterrupt: 

In [279]:
result.save_file("11. Voting Elephant NN cant learn kuznechik (9 errors, 4900 neurons)")

In [392]:
simple_song = Song([0, -1, 5, -1, 4, -1, 5, -1])
result = player.improvise(simple_song)
result.play()

In [393]:
simple_song = Song([0, -1, 5, -1, 4, -1, 7, -1])
result = player.improvise(simple_song)
result.play()